In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from io import StringIO

# Function to read and inspect CSV data
def read_and_inspect_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        required_columns = ['Atom', 'X', 'Y', 'Z']
        missing_columns = [col for col in required_columns if col not in df.columns]
        
        if missing_columns:
            raise ValueError(f"Missing columns: {', '.join(missing_columns)}")
        
        return df
    
    except pd.errors.ParserError as e:
        print(f"Error reading file with default settings: {e}.")
        return None
    except Exception as e:
        print(f"Error reading the file: {e}")
        return None

# Function to calculate dipole moment
def calculate_dipole_moment(df, charges):
    df['Charge'] = df['Atom'].apply(lambda atom: charges.get(atom.strip(), 0))
    mu_x = np.sum(df['Charge'] * df['X'])
    mu_y = np.sum(df['Charge'] * df['Y'])
    mu_z = np.sum(df['Charge'] * df['Z'])
    return np.array([mu_x, mu_y, mu_z])

# Function to automate reading from traj folders and calculating average dipole moments
def automate_dipole_moment_calculation(base_dir):
    charges_reactant = {
        'SG': -0.335, 'HG1': 0.155, 'CB': 0.06, 'HB1': 0.06, 'HB2': 0.06, 
        'NE2': -0.76, 'HE21': 0.38, 'HE22': 0.38, 
        'CD': 0.5, 'OE1': -0.5, 'HB2': 0.06, 'CG': -0.12
    }
    charges_product = {
        'SG': -0.9, 'HG1': 0.418, 'CB': -0.3, 'HB1': 0.1, 'HB2': 0.1, 
        'NE2': -0.4965, 'HE21': 0.48, 'HE22': 0.48, 
        'CD': 0.2858, 'OE1': -0.503, 'HB2': 0.06, 'CG': 0.2157
    }

    total_dipole_reactant = np.zeros(3)
    total_dipole_product = np.zeros(3)
    valid_traj_count = 0
    
    for i in range(100):
        traj_folder = os.path.join(base_dir, f'traj{i:02d}')
        reactant_file = os.path.join(traj_folder, 'reactant_qatoms.csv')
        product_file = os.path.join(traj_folder, 'product_qatoms.csv')
        
        if os.path.exists(reactant_file) and os.path.exists(product_file):
            print(f"Processing: {traj_folder}")
            
            df_reactant = read_and_inspect_csv(reactant_file)
            df_product = read_and_inspect_csv(product_file)
            
            if df_reactant is not None and df_product is not None:
                dipole_reactant = calculate_dipole_moment(df_reactant, charges_reactant)
                dipole_product = calculate_dipole_moment(df_product, charges_product)
                
                total_dipole_reactant += dipole_reactant
                total_dipole_product += dipole_product
                valid_traj_count += 1

                print(f"Reactant Dipole Moment (traj{i:02d}): {dipole_reactant}")
                print(f"Product Dipole Moment (traj{i:02d}): {dipole_product}\n")
            else:
                print(f"Skipping: {traj_folder} due to file read errors.")
        else:
            print(f"Skipping: {traj_folder} due to missing files.")
    
    if valid_traj_count > 0:
        # Calculate average dipole moments
        avg_dipole_reactant = total_dipole_reactant / valid_traj_count
        avg_dipole_product = total_dipole_product / valid_traj_count

        print(f"\nAverage Reactant Dipole Moment: {avg_dipole_reactant}")
        print(f"Average Product Dipole Moment: {avg_dipole_product}")
        
        # Calculate and print the angle between the average dipole moments
        norm_reactant = np.linalg.norm(avg_dipole_reactant)
        norm_product = np.linalg.norm(avg_dipole_product)
        
        dipole_reactant_normalized = avg_dipole_reactant / norm_reactant if norm_reactant != 0 else avg_dipole_reactant
        dipole_product_normalized = avg_dipole_product / norm_product if norm_product != 0 else avg_dipole_product

        dot_product = np.dot(dipole_reactant_normalized, dipole_product_normalized)
        angle_radians = np.arccos(np.clip(dot_product, -1.0, 1.0))
        angle_degrees = np.degrees(angle_radians)

        print(f"Angle Between Average Dipole Moments: {angle_degrees:.2f} degrees\n")
    else:
        print("No valid trajectories found.")

# Base directory containing traj folders
base_dir = "/home/hp/nayanika/github/PhD_Thesis/EVB/protein_stepwise/GPX6MUT/combination_4/mousecys/1-prep/47_48_52_54"

# Run the automation
automate_dipole_moment_calculation(base_dir)


Processing: /home/hp/nayanika/github/PhD_Thesis/EVB/protein_stepwise/GPX6MUT/combination_4/mousecys/1-prep/47_48_52_54/traj00
Reactant Dipole Moment (traj00): [ 7.46991868  6.09775515 11.94960177]
Product Dipole Moment (traj00): [ 4.59972909  1.52271604 10.10593393]

Processing: /home/hp/nayanika/github/PhD_Thesis/EVB/protein_stepwise/GPX6MUT/combination_4/mousecys/1-prep/47_48_52_54/traj01
Reactant Dipole Moment (traj01): [ 7.39381185  6.09100078 11.90182845]
Product Dipole Moment (traj01): [ 3.05804876  1.50221118 11.03475613]

Processing: /home/hp/nayanika/github/PhD_Thesis/EVB/protein_stepwise/GPX6MUT/combination_4/mousecys/1-prep/47_48_52_54/traj02
Reactant Dipole Moment (traj02): [ 7.34473923  6.1537471  12.04688602]
Product Dipole Moment (traj02): [ 3.825421    1.78411674 10.82718167]

Processing: /home/hp/nayanika/github/PhD_Thesis/EVB/protein_stepwise/GPX6MUT/combination_4/mousecys/1-prep/47_48_52_54/traj03
Reactant Dipole Moment (traj03): [ 7.37647686  5.99321929 11.88300044]